In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import interact
import ipywidgets as widgets

In [2]:
class BufferState(object):
    def __init__(self, Nmax, N):
        """
        Initialize Delay Block and Buffer state
        Nmax : Maximum delay possible
        N : Delay
        """

        M = 2**(math.ceil(np.log2(Nmax))+1)
        # Size of buffer, closest power of 2 that is at least 2 * Nmax

        Mmask = M-1
        # Masking value, for (mod M) opperation using bitand

        buff = np.zeros(M)        
        # Circular buffer

        self.Nmax = Nmax
        self.N = N
        self.M = M
        self.Mmask = Mmask
        self.buff = buff
        self.n_h = 0    # head of buffer
        self.n_t = N-1  # tail of buffer

    def delay(self, x):
        """
        Delays a signal by the specified number(state_in.N) of samples 
        state_in : input state
        x : input samples
        """
        if x.size > self.M:
            print("error: input bigger than buffer size")

        for i in range (x.size):
            #store a sample
            self.buff[self.n_t] = x[i]
            #increment tail index(circular)
            self.n_t = (self.n_t+1) & self.Mmask # s.n_t+1 % M
        
        # Get samples out of head to output
        y = np.zeros(x.size)  #output buffer of sample
        for i in range (y.size):
            #get a sample
            y[i] = self.buff[self.n_h]
            #increment head index
            self.n_h = (self.n_h+1) & self.Mmask # s.n_h+1 % M
        return y

In [5]:
#Testing delay block

# Fixed variables
Nb = 10 #number of buffers
Ns = 128 # samples in each buffer
Nmax = 200 # maximum delay


# generate random input samples / input signal
x = np.random.randn(Nb*Ns)

# reshape into buffers
xb = np.reshape(x, (Nb,Ns))

#Graph inital settings
n = np.arange(x.size).reshape((Nb*Ns)) # discrete time axis for graph
fig = go.FigureWidget()
fig.add_scatter(x = n, name = "delayed signal")
fig.add_scatter(x = n, y = x, name = "original signal")
fig.update_xaxes(title_text='n')
fig.update_yaxes(title_text='signal')

@interact(delay = (0,100,5))

def update(delay = 10):
    #Initialize the delay block
    state_delay1 = BufferState(Nmax, delay)

    # output samples
    yb = np.zeros((Nb, Ns))

    #process each buffer
    for i in range(Nb):
        yb[i] = state_delay1.delay(xb[i])
    y = np.reshape(yb, (Nb*Ns))

    with fig.batch_update():
        fig.data[0].y = y
fig

interactive(children=(IntSlider(value=10, description='delay', step=5), Output()), _dom_classes=('widget-inter…

FigureWidget({
    'data': [{'name': 'delayed signal',
              'type': 'scatter',
              'uid': '…

In [4]:
#Cascading two delay block

# Fixed variables
Nb = 10 #number of buffers
Ns = 128 # samples in each buffer
Nmax = 200 # maximum delay


# generate random input samples / input signal
x = np.random.randn(Nb*Ns)

# reshape into buffers
xb = np.reshape(x, (Nb,Ns))

#Graph inital settings
n = np.arange(x.size).reshape((Nb*Ns)) # discrete time axis for graph
fig = go.FigureWidget()
fig.add_scatter(x = n, y = x, name = "original signal")
fig.add_scatter(x = n, name = "1st delayed signal")
fig.add_scatter(x = n, name = "2nd delayed signal")
fig.update_xaxes(title_text='n')
fig.update_yaxes(title_text='signal')

@interact(delay1 = (0,100,5), delay2=(0,100,5))

def update(delay1 = 10, delay2 = 10):
    #Initialize the delay block
    state_delay1 = BufferState(Nmax, delay1)
    state_delay2 = BufferState(Nmax, delay2)

    # output samples
    yb1 = np.zeros((Nb, Ns))
    yb2 = np.zeros((Nb, Ns))

    #process each buffer
    for i in range(Nb):
        yb1[i] = state_delay1.delay(xb[i])
        yb2[i] = state_delay2.delay(yb1[i])
    y1 = np.reshape(yb1, (Nb*Ns))
    y2 = np.reshape(yb2, (Nb*Ns))

    with fig.batch_update():
        fig.data[1].y = y1
        fig.data[2].y = y2
fig

interactive(children=(IntSlider(value=10, description='delay1', step=5), IntSlider(value=10, description='dela…

FigureWidget({
    'data': [{'name': 'original signal',
              'type': 'scatter',
              'uid': …